In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model, datasets

In [2]:
# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("Running on GPU")
else:
    print("GPU not available, running on CPU")

Running on GPU


In [3]:
# Inception block equivalent in TensorFlow
class InceptionBlock(tf.keras.layers.Layer):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_pool):
        super(InceptionBlock, self).__init__()
        self.branch1 = layers.Conv2D(out_1x1, kernel_size=1, activation='relu')

        self.branch2 = tf.keras.Sequential([
            layers.Conv2D(red_3x3, kernel_size=1, activation='relu'),
            layers.Conv2D(out_3x3, kernel_size=3, padding='same', activation='relu')
        ])

        self.branch3 = tf.keras.Sequential([
            layers.Conv2D(red_5x5, kernel_size=1, activation='relu'),
            layers.Conv2D(out_5x5, kernel_size=5, padding='same', activation='relu')
        ])

        self.branch4 = tf.keras.Sequential([
            layers.MaxPool2D(pool_size=3, strides=1, padding='same'),
            layers.Conv2D(out_pool, kernel_size=1, activation='relu')
        ])

    def call(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        return tf.concat([branch1, branch2, branch3, branch4], axis=-1)


In [4]:

# GoogleNet model in TensorFlow
class GoogleNet(Model):
    def __init__(self, num_classes=10):
        super(GoogleNet, self).__init__()

        self.conv1 = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu')
        self.maxpool1 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        self.conv2 = layers.Conv2D(64, kernel_size=1, activation='relu')
        self.conv3 = layers.Conv2D(192, kernel_size=3, padding='same', activation='relu')
        self.maxpool2 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')

        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')

        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = layers.MaxPool2D(pool_size=3, strides=2, padding='same')

        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = layers.GlobalAveragePooling2D()
        self.dropout = layers.Dropout(0.4)
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = self.dropout(x)
        return self.fc(x)


In [5]:
# Data loading and preprocessing for CIFAR-10
def preprocess_data(x, y):
    x = tf.image.resize(x, (112, 112))  # Resize to match model's expected input size
    x = tf.image.per_image_standardization(x)
    return x, y

In [6]:
batch_size = 64
epochs = 10

In [7]:
(train_images, train_labels), (val_images, val_labels) = datasets.cifar10.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(preprocess_data).batch(batch_size).shuffle(10000)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(preprocess_data).batch(batch_size)

In [8]:
# Instantiate and compile the model
model = GoogleNet(num_classes=10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Model training
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 177s 162ms/step - accuracy: 0.2144 - loss: 2.0273 - val_accuracy: 0.4299 - val_loss: 1.5082
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 123s 98ms/step - accuracy: 0.4874 - loss: 1.3875 - val_accuracy: 0.5979 - val_loss: 1.1204
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 77ms/step - accuracy: 0.6253 - loss: 1.0500 - val_accuracy: 0.6696 - val_loss: 0.9332
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 86s 81ms/step - accuracy: 0.6913 - loss: 0.8633 - val_accuracy: 0.6836 - val_loss: 0.9193
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 78ms/step - accuracy: 0.7422 - loss: 0.7368 - val_accuracy: 0.7224 - val_loss: 0.8089
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 78ms/step - accuracy: 0.7765 - loss: 0.6446 - val_accuracy: 0.7435 - val_loss: 0.7516
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 80ms/step - accuracy: 0.7981 - loss: 0.5831 - val_accuracy: 0.7468 - val_loss: 0.7649
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 71s 81ms/step - accuracy: 0.8217 - loss: 0.5124